In [ ]:
import statsmodels
import statsmodels.api as sm
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.api import ExponentialSmoothing, SimpleExpSmoothing, Holt

In [ ]:
hobbies_1_index = [c for c in past_sales.columns if 'HOBBIES_1' in c]
hobbies_2_index = [c for c in past_sales.columns if 'HOBBIES_2' in c]
household_1_index = [c for c in past_sales.columns if 'HOUSEHOLD_1' in c]
household_2_index = [c for c in past_sales.columns if 'HOUSEHOLD_2' in c]
food_1_index = [c for c in past_sales.columns if 'FOOD_1' in c]
food_2_index = [c for c in past_sales.columns if 'FOOD_2' in c]
food_3_index = [c for c in past_sales.columns if 'FOOD_3' in c]

In [ ]:
CA_index = [c for c in past_sales.columns if 'CA' in c]
TX_index = [c for c in past_sales.columns if 'TX' in c]
WI_index = [c for c in past_sales.columns if 'WI' in c]
CA_1_index = [c for c in past_sales.columns if 'CA_1' in c]
CA_2_index = [c for c in past_sales.columns if 'CA_2' in c]
CA_3_index = [c for c in past_sales.columns if 'CA_3' in c]
CA_4_index = [c for c in past_sales.columns if 'CA_4' in c]
TX_1_index = [c for c in past_sales.columns if 'TX_1' in c]
TX_2_index = [c for c in past_sales.columns if 'TX_2' in c]
TX_3_index = [c for c in past_sales.columns if 'TX_3' in c]
WI_1_index = [c for c in past_sales.columns if 'WI_1' in c]
WI_2_index = [c for c in past_sales.columns if 'WI_2' in c]
WI_3_index = [c for c in past_sales.columns if 'WI_3' in c]

In [ ]:
I tried to see all 1913 dates, but the number of data was too large, so I randomly selected 100 days and visualized them.<br>
1913개의 날짜를 다 보려고 하니 데이터의 수가 너무 많으므로, 임의로 100일을 골라 시각화하였습니다.

In [ ]:
ids = sorted(list(set(sales_train_val['id'])))
d_cols = [c for c in sales_train_val.columns if 'd_' in c]
x_1 = sales_train_val.loc[sales_train_val['id']==ids[5000]].set_index('id')[d_cols].values[0][1500:1600]
x_2 = sales_train_val.loc[sales_train_val['id']==ids[15000]].set_index('id')[d_cols].values[0][800:900]
x_3 = sales_train_val.loc[sales_train_val['id']==ids[25000]].set_index('id')[d_cols].values[0][1500:1600]

fig = make_subplots(rows=3, cols=1)

fig.add_trace(go.Scatter(x=np.arange(len(x_1))+1500, y=x_1, showlegend=False,
                        mode='lines+markers', name='First Sample', marker=dict(color='chartreuse')),
             row=1, col=1)

fig.add_trace(go.Scatter(x=np.arange(len(x_2))+800, y=x_2, showlegend=False,
                        mode='lines+markers', name='Second Sample', marker=dict(color='violet')),
             row=2, col=1)

fig.add_trace(go.Scatter(x=np.arange(len(x_3))+1600, y=x_3, showlegend=False,
                        mode='lines+markers', name='Third Sample', line=dict(color='yellow')),
             row=3, col=1)

fig.update_layout(height=1200, width=800, title_text='Sample sales snippets', template='plotly_dark')
fig.show()

### 2.3 Denoising
As we mentioned earlier, we can check that the sales data is very fluctuating and volatile. Sometimes, the sales are zero for a few days in a row, and at other times, it remains at its peak value for a few days. Therefore, we need some sort of "denoising" techniques to find the underlying trends in the sales data and make forecasts.

In [ ]:
def maddest(d, axis=None):
    return np.mean(np.absolute(d - np.mean(d, axis)), axis)

def denoise_signal(x, wavelet='db4', level=1):
    coeff = pywt.wavedec(x, wavelet, mode="per")
    sigma = (1/0.6745) * maddest(coeff[-level])

    uthresh = sigma * np.sqrt(2*np.log(len(x)))
    coeff[1:] = (pywt.threshold(i, value=uthresh, mode='hard') for i in coeff[1:])

    return pywt.waverec(coeff, wavelet, mode='per')

In [ ]:
y_w1 = denoise_signal(x_1)
y_w2 = denoise_signal(x_2)
y_w3 = denoise_signal(x_3)


fig = make_subplots(rows=3, cols=1)

fig.add_trace(go.Scatter(x=np.arange(len(x_1)), y=x_1, showlegend=False, mode='lines+markers',
                        name='Original Signal', marker=dict(color='chartreuse')),
             row=1, col=1)
fig.add_trace(go.Scatter(x=np.arange(len(x_1)), y=y_w1, showlegend=False, mode='lines',
                        name='Denoised Signal', marker=dict(color='cornsilk')),
             row=1, col=1)

fig.add_trace(go.Scatter(x=np.arange(len(x_2)), y=x_2, showlegend=False, mode='lines+markers',
                        name='Original Signal', marker=dict(color='violet')),
             row=2, col=1)
fig.add_trace(go.Scatter(x=np.arange(len(x_2)), y=y_w2, showlegend=False, mode='lines',
                        name='Denoised Signal', marker=dict(color='cornsilk')),
             row=2, col=1)

fig.add_trace(go.Scatter(x=np.arange(len(x_2)), y=x_2, showlegend=False, mode='lines+markers',
                        name='Original Signal', marker=dict(color='lightskyblue')),
             row=3, col=1)
fig.add_trace(go.Scatter(x=np.arange(len(x_2)), y=y_w2, showlegend=False, mode='lines',
                        name='Denoised Signal', marker=dict(color='cornsilk')),
             row=3, col=1)

fig.update_layout(height=1200, width=800, 
                  title_text="Original(Pale) sales vs. Denoised(Dark) sales", template='plotly_dark')
fig.show()

In [ ]:
fig, ax = plt.subplots(nrows=3, ncols=2, figsize=(30, 20))

ax[0, 0].plot(x_1, color='chartreuse', marker='o')
ax[0, 0].set_title('Original Sales', fontsize=30)
ax[0, 1].plot(y_w1, color='cyan', marker='.')
ax[0, 1].set_title('After Wavelet Denoising', fontsize=30)

ax[1, 0].plot(x_2, color='chartreuse', marker='o')
ax[1, 0].set_title('Original Sales', fontsize=30)
ax[1, 1].plot(y_w2, color='cyan', marker='.')
ax[1, 1].set_title('After Wavelet Denoising', fontsize=30)

ax[2, 0].plot(x_3, color='chartreuse', marker='o')
ax[2, 0].set_title('Original Sales', fontsize=30)
ax[2, 1].plot(y_w3, color='cyan', marker='.')
ax[2, 1].set_title('After Wavelet Denoising', fontsize=30)

plt.show()

### 2.4 Average Smoothing
Average smooting is a relatively simple way to denoise time series data. In this method, we take a "window" with a fixed size. We first place the window at the beginning of the time series (first ten elements) and calculate the mean of that section. We now move the window across the time series in the forward direction by a particular "stride", calculate the mean of the new window and repeat the process, until we reach the end of the time series. All the mean values we calculated are then concatenated into a new time series, which forms the denoised sales data.

In [ ]:
def average_smoothing(signal, kernel_size=3, stride=1):
    sample = []
    start = 0
    end = kernel_size
    while end <= len(signal):
        start += stride
        end += stride
        sample.extend(np.ones(end - start)*np.mean(signal[start:end]))
    return np.array(sample)

In [ ]:
y_a1 = average_smoothing(x_1)
y_a2 = average_smoothing(x_2)
y_a3 = average_smoothing(x_3)

fig, ax = plt.subplots(nrows=3, ncols=2, figsize=(30, 20))

ax[0, 0].plot(x_1, color='chartreuse', marker='o')
ax[0, 0].set_title('Original Sales', fontsize=30)
ax[0, 1].plot(y_a1, color='cyan', marker='.')
ax[0, 1].set_title('After Wavelet Denoising', fontsize=30)

ax[1, 0].plot(x_2, color='chartreuse', marker='o')
ax[1, 0].set_title('Original Sales', fontsize=30)
ax[1, 1].plot(y_a2, color='cyan', marker='.')
ax[1, 1].set_title('After Wavelet Denoising', fontsize=30)

ax[2, 0].plot(x_3, color='chartreuse', marker='o')
ax[2, 0].set_title('Original Sales', fontsize=30)
ax[2, 1].plot(y_a3, color='cyan', marker='.')
ax[2, 1].set_title('After Wavelet Denoising', fontsize=30)

plt.show()